# Exercise 4 - Part 2: Working with binary files

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Problem 4: Parsing a binary file 

This homework will review and combine multiple concepts:
- file handling with focus on a binary file
- data types: conversion between types, and manipulation of data
- `numpy.array` data type: creation, manipulation, and slicing
- `matplotlib` extension for showing images

**Given**: 
- a binary file named `612A8347.ppm` (_download from the class website_)
- jpg file named `612A8347.jpg` to show you how the final result should look like (_for validation only_)
- a description of the binary file (given after the task list)

**Your tasks**: 
1. read this file making sure to implement at least some suitable exception handling.
   Break up the task in 
   1. reading and analyzing the general information 
   1. identifying the correct data type and format string from that information
   1. reading image information into a three-dimensional `numpy.array` named `data`.  
      `data` is organized as a 2-d array of RGB pixels, i.e., each `data[i,j,:]` is an `array([R,G,B])` for R (red), G (green), and B (blue) color values, all in the range $0\le R,G,B \le 1$.
2. Using `matplotlib.pyplot.imshow(...)`, plot the image contained in your data.
3. Create and plot three RGB-images, each containing only one or the three color channels, and plot them side by side.
4. For each of the color channels, create and plot a histogram with 256 equally sized bins (check out `numpy` and `matplotlib` for ways to generate and visualize histograms).
5. Break the original RBG image into 3x3 = 9 equal images and plot those.  Use python's `slicing` (see both `lists` and `numpy.array` documentations for that).

### Color Depth

RGB -> 8bit per color (= 1 byte per color) -> $2^{8} = 256$

RGB -> 16bit per color (= 2 byte per color) -> $2^{16} = 256^2 = 65636$

### locate your image.  Mine is placed inside a folder named `images` inside the `jupyter-notebooks` folder.

In [2]:
%ls

00 Outline.ipynb
01 Basic Concepts.ipynb
01.1 Exercise_Functions.ipynb
01.2 Solution of Exercise_Functions.ipynb
02 Matrix Data Types.ipynb
02.1 Exercise numpy.ipynb
02.2 Solution of Exercise numpy.ipynb
02.3 Exercise on more matrix operations.ipynb
02.4 Solution for Exercise on more matrix operations.ipynb
03 Plotting using matplotlib.ipynb
03.1 Exercise Plotting.ipynb
03.2 Solution of Exercise Plotting.ipynb
04 File handling.ipynb
04.1 Exercise File handling.ipynb
04.2 Solution of Exercise File handling.ipynb
04.3 Exercise Handling a binary file.ipynb
04.4 Solution of Exercise Handling a binary file.ipynb
05 Classes.ipynb
05.1 Exercise Classes.ipynb
05.2 Solution of Exercise Classes.ipynb
06 Modules.ipynb
06.1 Exercise Modules.ipynb
06.2 Solution of Exercise Modules.ipynb
07 ML/
07 ML_basics.ipynb
07 Machine Learning AI.ipynb
08 User Interface with PyQt5.ipynb
09 SQL Databases.ipynb
10 Web Scrabbing.ipynb
80 IPython Widgets.ipynb
90 Exploring special features of jupyter notebooks.ipy

In [3]:
%ll images

total 168288
-rw-r--r--@ 1 pmackenz  staff   5269451 Oct 22 15:01 612A8347.jpg
-rw-r--r--@ 1 pmackenz  staff  39992995 Oct 22 15:04 612A8347.ppm
-rw-r--r--@ 1 pmackenz  staff  40010224 Oct 22 15:01 612A8347.tif


### This example code explores the file byte by byte and reads and displays the first 24 characters.

This should help you get started with reading the header information

In [4]:
# You may need to change the filename if you are storing the image in a different location
source_filename = 'images/612A8347.ppm'

try:
    f=open(source_filename,'rb')
except IOError:
    print(f'CANNOT OPEN {source_filename} FOR READING')
    raise
    
for i in range(24):
    c = f.read(1)
    print("c[{}]: {}".format(i,c))

f.close()

c[0]: b'P'
c[1]: b'6'
c[2]: b'\n'
c[3]: b'3'
c[4]: b'1'
c[5]: b'6'
c[6]: b'2'
c[7]: b' '
c[8]: b'2'
c[9]: b'1'
c[10]: b'0'
c[11]: b'8'
c[12]: b'\n'
c[13]: b'6'
c[14]: b'5'
c[15]: b'5'
c[16]: b'3'
c[17]: b'5'
c[18]: b'\n'
c[19]: b'\xad'
c[20]: b'u'
c[21]: b'\xbc'
c[22]: b'.'
c[23]: b'\xd1'


1.1 **Read the header information**

From the above initial attempt, we conclude the following:

- The binary file starts with human readable information
- There are three (3) lines of information, each ending with `'\n'`
    1. the first line contains a string, known as `magic number`.  You may google for that magic number to ID the file type.  We may ignore it for this exercise.
    2. the second line contains two numbers: width and height of the stored image in pixels.
    3. the number of colors per channel or `color depth`.  

In [ ]:
import sys
from struct import unpack, calcsize
import numpy as np

source_filename = 'images/612A8347.ppm'

# YOUR CODE HERE ...

print(magic_number)
print(size)
print(color_depth)


1.2. **Find dimension of the data from `color_depth`**

The third number represented color depth. 
That number is $2^{bits}-1$, with $bits$ as the  available number of bits in the data type used to represent each channel. **Note**: each byte holds 8 bits.  Common data types at `unsigned byte`, `unsigned character`, `unsigned short`, `unsigned integer`, `undigned long` or `unsigned long long`.  Use `struct.calcsize(fmt)` to identify which of the data types matches the given color depth.

**References**:
- https://docs.python.org/2/library/struct.html#format-characters

In [ ]:
# find bytes per color
N = 0
fmt = ''

candidates = ['B', ... ]    # YOUR list of suitable candidates

for t in candidates:
    
    # YOUR CODE HERE ...
    
    N = ...
    fmt = ...
        
        
print(f"---\nthe suggested format is '{fmt}' and takes {N} bytes per value")

1.3. **Read the pixels into a 3-dimensional `numpy.array` named `data`**

Now that you know the correct data type for the binary storage scheme, read the data as follows:
- each pixel is represented by three consecutive numbers of the identified data type.  These numbers, in sequence, represent R, G, and B values.  Those values are $0\le R,G,B\le color~depth$ and nned to be converted to `np.array([r,g,b])` with $0\le r,g,b\le 1$.
- pixels are stored in the file sequentially forming row after row of your image.  You should know the size from the header data.
- make sure the colleacted data is a `numpy.array`!
- **Hint**: depending on your operation system, the data may appear in a native or non-native format.  You may need to specify whether your binary data is to be interpreted as `little-endian` or `big-endian` (see documentation for `struct`)

In [ ]:
# read the data from file
data = []

FMT = ... # the format string for one pixel (RGB values, byte order)


# YOUR CODE HERE ...

    
# we are done reading that file
f.close()

In [ ]:
# check the shape of data.  The answer should be (num_rows, num_cols, 3)
data.shape

2. **Visualize the data as RGB image**

- Check out `matplotlib.pyplot.imshow(...)`.  It should be able to visualize your `data` array quickly as an image.
- **Hint**: depending on your operation system, the data may appear in a native or non-native format.  You may need to specify whether your binary data is to be interpreted as `little-endian` or `big-endian` (see documentation for `struct`).  Picking the wrong one will make the image look very weird, while picking the write one will disclose what I've done last weekend.

In [ ]:
import matplotlib.pyplot as plt

# YOUR CODE HERE ...


3.1 **Visualize each color channel seperately**

While it may be tempting to use `data[:,:,i]` for the `i`th channel, that form represents a greysacle image of that color channel and will generate fake colors.  Make sure to create another RGB image.

In [ ]:

# YOUR CODE HERE ...


3.2 **Here is the original image**

This is for validation only

In [ ]:
img = mpimg.imread(source_filename[:-3]+'jpg')
plt.imshow(img)

4. **Generate histograms for each color channel**

**Useful options**:
- `numpy.histogram`: https://numpy.org/doc/stable/reference/generated/numpy.histogram.html
- `matplotlib.pyplot.hist`: https://numpy.org/doc/stable/reference/generated/numpy.histogram.html

In [ ]:

# YOUR CODE HERE ...


5. **Viewing a zoomed image**

Think of slicing that image both vertically and horizontally into nine (9) equal images. Plot those 9 images in a 3x3 matrix (`matplotlib.pyplot.subplots`)

In [ ]:
fig,ax = plt.subplots(3,3)


# YOUR CODE HERE ...
